
# Pokémon Math Mystery (Checkerboard Adventure)
*A calm, story-first multiplication & division puzzle game built for Connor.*

**What it is:**  
- Move your creature across a checkerboard to reach the **GYM**.  
- Each move opens a math prompt (no racing or leaderboards).  
- Use **Number Line**, **Equal Groups**, and **Fact Web** hints displayed at the bottom.  
- Collect **clue letters** to solve a simple mystery phrase at the end.

**How to use images/gifs:**  
- Put images into the `assets/` folder (next cell makes it for you).  
  - Example names you can use in the code below: `pikachu.png`, `charmander.gif`, `squirtle.png`, `bulbasaur.png`.  
  - Or use your own names (e.g., `my_creature.png`) and select it in the **Token Image** dropdown.
- If an image isn't found, the game uses emoji tokens so it always works.




In [ ]:

import os
ASSETS_DIR = "assets"
os.makedirs(ASSETS_DIR, exist_ok=True)

with open(os.path.join(ASSETS_DIR, "README.txt"), "w") as f:
    f.write(
        "Drop your PNG/JPG/GIF images here. Example filenames that the notebook references:\n"
        "- pikachu.png\n- charmander.gif\n- squirtle.png\n- bulbasaur.png\n\n"
        "You can add any filenames you want; just select them in the Token Image dropdown.\n"
    )

print("Assets folder ready at:", ASSETS_DIR)


Assets folder ready at: assets


In [ ]:

import math
import random
from typing import List, Tuple, Dict, Optional

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, HTML, Markdown
import ipywidgets as W
from pathlib import Path
from PIL import Image, features,  ImageSequence

# --- Image support (PNG/JPG/GIF/WEBP) --------------------------------------
WEBP_ENABLED = features.check("webp")

def load_token_image(path) -> Optional[Image.Image]:
    """
    Returns an RGBA Pillow image or None (on error/missing/unsupported).
    - Opens PNG/JPG/WEBP.
    - If GIF/WEBP is animated, takes the first frame.
    - If WEBP isn't supported by this Pillow build, prints a friendly note.
    """
    p = Path(path)
    if not p.exists():
        return None
    try:
        im = Image.open(p)
        # If animated (gif/webp), use first frame
        if getattr(im, "is_animated", False):
            im = next(ImageSequence.Iterator(im))
        return im.convert("RGBA")
    except Exception as e:
        # Helpful message for WEBP without support
        if p.suffix.lower() == ".webp" and not WEBP_ENABLED:
            print("WEBP not supported by this Pillow build. Try: pip install --upgrade pillow")
        else:
            print(f"Image load error for {p.name}: {e}")
        return None

# Simple utility
def rand_item(xs):
    return xs[random.randrange(len(xs))]

def clamp(x, lo, hi):
    return max(lo, min(hi, x))


In [ ]:

# --- Story & Config --------------------------------------------------------

# Chapters and a light 'mystery' phrase; each correct answer yields one letter.
MYSTERY_PHRASES = [
    "TRAINER BADGE",
    "ELECTRIC ENERGY",
    "MYSTERY GYM",
    "POCKET CREATURES",
]

STORY_BEATS = [
    "A quiet morning in Pal Meadow. Your partner creature nudges you awake — the town's Gym is glowing with strange symbols.",
    "At the meadow gate, you find a note: 'Only those who understand equal groups may pass...'",
    "A breeze flips another card: 'Count your jumps on the number line to cross the river.'",
    "The Gym doors shimmer: 'Know a fact, and its family follows.'",
    "Inside the entry hall, a pedestal asks for a **mystery phrase**. Each correct step will reveal a letter...",
]

DEFAULT_FACTORS = [0, 1, 2, 5, 10]  # start aligned with class

size_dropdown = W.Dropdown(options=[6,7,8,9,10], value=8, description="Board")
mode_dropdown = W.Dropdown(options=[("Multiplication", "multiply"), ("Division", "divide"), ("Mixed", "mixed")],
                           value="mixed", description="Mode")
factors_text = W.Text(value="0,1,2,5,10", description="Factors", layout=W.Layout(width="300px"))
phrase_dropdown = W.Dropdown(options=MYSTERY_PHRASES, value=MYSTERY_PHRASES[0], description="Mystery")
show_numline = W.Checkbox(value=True, description="Show Number Line")
show_groups  = W.Checkbox(value=True, description="Show Equal Groups")
show_factweb = W.Checkbox(value=True, description="Show Fact Web")

# Token choices: emoji fallback + optional image file name from /assets
EMOJI_TOKENS = ["⚡","🔥","💧","🍃","⭐","🌙","🪨","🌀"]
def list_asset_images():
    p = Path("/mnt/data/assets")
    if not p.exists():
        return []
    return sorted([f.name for f in p.iterdir() if f.suffix.lower() in {".png",".jpg",".jpeg",".gif",".webp"}])

token_type = W.Dropdown(options=["Emoji","Image"], value="Emoji", description="Token Type")
token_emoji = W.Dropdown(options=EMOJI_TOKENS, value="⚡", description="Emoji")
token_image = W.Dropdown(options=(["(none)"] + list_asset_images()), value="(none)", description="Token Image")

ui = W.VBox([
    W.HTML("<b>Game Setup</b>"),
    W.HBox([size_dropdown, mode_dropdown]),
    W.HBox([factors_text, phrase_dropdown]),
    W.HBox([show_numline, show_groups, show_factweb]),
    W.HBox([token_type, token_emoji, token_image]),
])
display(ui)


In [ ]:

# --- Game State ------------------------------------------------------------

class Problem:
    def __init__(self, a, b, op, prompt, answer, dividend=None, divisor=None):
        self.a = a
        self.b = b
        self.op = op
        self.prompt = prompt
        self.answer = answer
        self.dividend = dividend
        self.divisor = divisor

def parse_factors(text):
    xs = []
    for part in text.split(","):
        part = part.strip()
        if not part:
            continue
        try:
            n = int(part)
            if 0 <= n <= 12:
                xs.append(n)
        except:
            pass
    return xs or DEFAULT_FACTORS

def make_problem(factors, mode):
    if mode == "multiply":
        a = rand_item(factors)
        b = rand_item(factors)
        return Problem(a,b,"×", f"{a} × {b} = ?", a*b)
    if mode == "divide":
        a = rand_item(factors); b = rand_item(factors)
        dividend = a*b
        divisor = rand_item([a,b])
        answer = dividend // divisor if divisor else 0
        return Problem(a,b,"÷", f"{dividend} ÷ {divisor} = ?", answer, dividend, divisor)
    # mixed
    return make_problem(factors, "multiply" if random.random()<0.5 else "divide")

class Game:
    def __init__(self):
        self.size = size_dropdown.value
        self.start = (self.size-1, 0)
        self.goal  = (0, self.size-1)
        self.pos   = self.start
        self.visited = {self.start}
        self.factors = parse_factors(factors_text.value)
        self.mode = mode_dropdown.value
        self.attempts = 0
        self.correct  = 0
        self.clue_chars = []  # revealed letters
        self.phrase = phrase_dropdown.value
        self.msg = "Move to an adjacent square to start your adventure."
        self.problem: Optional[Problem] = None

    def reset(self):
        self.__init__()

    def is_adjacent(self, a, b):
        return (abs(a[0]-b[0])==1 and a[1]==b[1]) or (abs(a[1]-b[1])==1 and a[0]==b[0])

    def open_problem_for(self, tile):
        if not self.is_adjacent(self.pos, tile):
            self.msg = "Pick a square next to your creature."
            return
        self.target = tile
        self.problem = make_problem(self.factors, self.mode)

    def submit(self, user_answer: str):
        if not self.problem:
            return False
        self.attempts += 1
        try:
            val = int(user_answer)
        except:
            self.msg = "Enter a whole number."
            return False

        if val == self.problem.answer:
            self.correct += 1
            self.pos = self.target
            self.visited.add(self.pos)
            self.problem = None

            # Reveal a new clue character from the secret phrase (skip spaces)
            remaining = [ch for ch in self.phrase if (ch != " " and ch not in self.clue_chars)]
            if remaining:
                self.clue_chars.append(rand_item(remaining))
            if self.pos == self.goal:
                self.msg = "🎉 You reached the Gym! Can you solve the mystery phrase below?"
            else:
                self.msg = rand_item([
                    "Nice! Your partner cheers quietly.",
                    "That fact is yours now.",
                    "You feel more confident — onward.",
                    "A gentle breeze reveals a new glyph on the path.",
                ])
            return True
        else:
            self.msg = "Close! Peek at a hint, then try again."
            return False

G = Game()


In [ ]:

# --- Board Visualization ---------------------------------------------------

def draw_board(g: Game, token_img=None, token_emoji="⚡"):
    n = g.size
    fig, ax = plt.subplots(figsize=(5,5))
    # Create a checkerboard with numeric array; imshow with default colormap
    board = np.add.outer(np.arange(n), np.arange(n)) % 2
    ax.imshow(board, interpolation='nearest')
    ax.set_xticks(range(n)); ax.set_yticks(range(n))
    ax.set_xticklabels([]); ax.set_yticklabels([])
    ax.grid(False)
    # Mark START and GYM text
    ax.text(0, n-1, "GYM", ha="center", va="center", fontsize=10)
    ax.text(0, 0, "", ha="center", va="center", fontsize=10)
    ax.text(n-1, 0, "START", ha="center", va="center", fontsize=10)

    # Place token at g.pos
    r, c = g.pos
    if token_img is not None:
        # draw image at (c, r)
        extent = [c-0.4, c+0.4, r+0.4, r-0.4]  # left, right, bottom, top
        ax.imshow(token_img, extent=extent)
    else:
        ax.text(c, r, token_emoji, ha="center", va="center", fontsize=22)

    plt.show()


In [ ]:

# --- Hints -----------------------------------------------------------------

def number_line_steps(prob: Problem):
    if prob.op == "×":
        step = prob.a; times = prob.b
        return [(i+1)*step for i in range(times)]
    else:  # dividend ÷ divisor
        step = prob.divisor if prob.divisor else 1
        target = prob.dividend if prob.dividend else 0
        jumps = max(1, target // max(1, step))
        return [(i+1)*step for i in range(jumps)]

def show_number_line(prob: Problem):
    steps = number_line_steps(prob)
    if not steps:
        display(Markdown("_No steps to show._"))
        return
    maxv = steps[-1]
    ticks = max(5, min(10, math.ceil(maxv/2)))
    xs = np.linspace(0, maxv, ticks+1)
    fig, ax = plt.subplots(figsize=(5,1.5))
    ax.hlines(0, 0, maxv)
    for x in xs:
        ax.vlines(x, -0.05, 0.05)
        ax.text(x, -0.15, str(int(round(x))), ha="center", va="top", fontsize=8)
    last = 0
    for s in steps:
        ax.hlines(0, last, s, linewidth=3)
        ax.text(s, 0.1, str(s), ha="center", va="bottom", fontsize=8)
        last = s
    ax.set_ylim(-0.3, 0.3); ax.set_xlim(0, maxv)
    ax.axis("off")
    plt.show()

def show_equal_groups(prob: Problem):
    if prob.op == "×":
        groups, in_each = prob.a, prob.b
    else:
        groups, in_each = (prob.divisor or 1), prob.answer

    # Draw small boxes (no specific colors)
    cols = min(5, groups)
    rows = math.ceil(groups/cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols*1.6, rows*1.6))
    if not isinstance(axes, np.ndarray):
        axes = np.array([[axes]])
    axes = axes.flatten()
    for g in range(groups):
        ax = axes[g]
        ax.set_title(f"Group {g+1}", fontsize=8)
        # draw in_each dots as small squares
        side = math.ceil(math.sqrt(in_each))
        grid = np.zeros((side, side))
        ax.imshow(grid)
        ax.set_xticks([]); ax.set_yticks([])
        # overlay count text
        ax.text(0.5, 0.5, f"{in_each}", transform=ax.transAxes, ha="center", va="center", fontsize=10)
    for k in range(groups, len(axes)):
        axes[k].axis("off")
    plt.tight_layout()
    plt.show()

def fact_web(prob: Problem):
    if prob.op == "×":
        a, b = prob.a, prob.b; p = a*b
        facts = [f"{a} × {b} = {p}", f"{b} × {a} = {p}", f"{p} ÷ {a} = {b}", f"{p} ÷ {b} = {a}"]
    else:
        d, v, q = prob.dividend, prob.divisor, prob.answer
        facts = [f"{q} × {v} = {d}", f"{v} × {q} = {d}", f"{d} ÷ {v} = {q}", f"{d} ÷ {q} = {v}"]
    display(Markdown("**Fact Web**  \n" + " · ".join(facts)))


In [ ]:

# --- Interaction -----------------------------------------------------------

move_msg = W.HTML("<i>Click an arrow to try moving.</i>")
ans = W.Text(placeholder="Enter your answer")
submit_btn = W.Button(description="Submit", button_style="primary")
cancel_btn = W.Button(description="Cancel")
refresh_btn = W.Button(description="New Board")

# Movement buttons
up_btn = W.Button(description="↑"); down_btn = W.Button(description="↓")
left_btn = W.Button(description="←"); right_btn = W.Button(description="→")

status = W.HTML("")
story = W.HTML("")
clue_out = W.HTML("")

board_out = W.Output()
hints_out = W.Output()

def token_image_or_none():
    if token_type.value == "Image" and token_image.value != "(none)":
        path = Path("/mnt/data/assets")/token_image.value
        if path.exists():
            try:
                im = Image.open(path).convert("RGBA")
                return im
            except Exception as e:
                print("Image load error:", e)
    return None

def refresh_all():
    board_out.clear_output(wait=True)
    with board_out:
        draw_board(G, token_img=token_image_or_none(), token_emoji=token_emoji.value)

    # Update text chunks
    status.value = f"<b>Correct:</b> {G.correct} · <b>Attempts:</b> {G.attempts}<br/>{G.msg}"
    # Mystery phrase reveal
    revealed = "".join([ch if ch in G.clue_chars or ch==' ' else "•" for ch in G.phrase])
    clue_out.value = f"<b>Mystery Phrase:</b> <span style='font-family:monospace'>{revealed}</span>"
    # Story beat
    idx = min(len(G.visited)-1, len(STORY_BEATS)-1)
    story.value = f"<i>{STORY_BEATS[idx]}</i>"

def try_move(dr, dc):
    r, c = G.pos
    nr, nc = clamp(r+dr, 0, G.size-1), clamp(c+dc, 0, G.size-1)
    if (nr, nc) == G.pos:
        G.msg = "Edge of the board. Try another direction."
        refresh_all()
        return
    if not G.is_adjacent(G.pos, (nr, nc)):
        G.msg = "Pick a neighboring square."
        refresh_all()
        return
    G.open_problem_for((nr, nc))
    if G.problem:
        # show a minimal 'prompt' UI
        with hints_out:
            hints_out.clear_output(wait=True)
            display(Markdown(f"### Solve to move: **{G.problem.prompt}**"))
            if show_numline.value:
                display(Markdown("**Number Line**"))
                show_number_line(G.problem)
            if show_groups.value:
                display(Markdown("**Equal Groups**"))
                show_equal_groups(G.problem)
            if show_factweb.value:
                fact_web(G.problem)

        ans.value = ""
        ans.focus()

def on_submit(_):
    ok = G.submit(ans.value.strip())
    refresh_all()
    # Clear hints after a correct move
    if ok:
        hints_out.clear_output(wait=True)

def on_cancel(_):
    G.problem = None
    G.msg = "Okay — move when you're ready."
    hints_out.clear_output(wait=True)
    refresh_all()

def on_new_board(_):
    G.size = size_dropdown.value
    G.factors = parse_factors(factors_text.value)
    G.mode = mode_dropdown.value
    G.phrase = phrase_dropdown.value
    G.reset()
    refresh_all()

submit_btn.on_click(on_submit)
cancel_btn.on_click(on_cancel)
refresh_btn.on_click(on_new_board)

up_btn.on_click(lambda _: try_move(-1, 0))
down_btn.on_click(lambda _: try_move(1, 0))
left_btn.on_click(lambda _: try_move(0, -1))
right_btn.on_click(lambda _: try_move(0, 1))

controls = W.VBox([
    W.HTML("<hr><b>Play</b>"),
    W.HBox([up_btn, down_btn, left_btn, right_btn]),
    W.HBox([ans, submit_btn, cancel_btn, refresh_btn]),
    status, story, clue_out,
])
display(W.HBox([W.VBox([controls, hints_out], layout=W.Layout(width="55%")), board_out]))
on_new_board(None)



## Tips for Parents
- Keep sessions short (10–15 min). Let your child steer the pace.
- Toggle hints as needed, then slowly hide them as facts become automatic.
- Add your own images/gifs in `assets/`, then choose them in **Token Type = Image**.
- You can edit the `MYSTERY_PHRASES` list at the top to personalize the final reveal.
- If multiplication is comfy, switch **Mode** to *Division* or *Mixed*.
- Celebrate finishing the mystery phrase — not speed!